In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_output import run_solver_output
from solver_output import run_solver_shift

## Copied and Pasted from Alberto's readme


The solver is a single Python script, contained in `solver/solver.py`.
It takes the following parameters:
* `-m` to specify the model name (`base`, `fixed`, `partflex`, or `flex`).
* `-i` to specify the location of the instance file.
* `-c` to specify the outsourcing cost multiplier (parameter OC in the paper).
* `-r` to specify the regional bound multiplier (parameter RM in the paper).
* `-g` to specify the global bound multiplier (parameter GM in the paper).
* `-u` to specify the maximum number of shift start times for the `partflex` model (parameter $\mu$ in the paper).
* `-o` to specify the location of the JSON solution file produced by the solver.

In [12]:
#get all the extra parameter information

df_complete = pd.read_csv(r'/home/clarice/Documents/VSCode/Term3/Masters_Thesis/lmd-sizing-scheduling/results/complete.csv')

print(df_complete['OC'].unique().tolist())
print(df_complete['RM'].unique().tolist())
print(df_complete['GM'].unique().tolist())


[1.2, 1.5, 1.8, 2.0, 2.5]
[0.75, 1.0, 1.5, 3.0, 5.0]
[0.6, 0.7, 0.8, 0.9, 1.0]


instance                                    object
model                                       object
city                                        object
DB                                         float64
DT                                          object
OC                                         float64
RM                                         float64
GM                                         float64
num_periods                                  int64
num_scenarios                                int64
obj_value                                  float64
elapsed_time                               float64
n_variables                                  int64
n_constraints                                int64
n_nonzeroes                                  int64
hiring_costs                               float64
outsourcing_costs                          float64
regional_avg_hired_pct                     float64
global_avg_hired_pct                       float64
n_shift_start_periods          

In [3]:
#Export the raw schduling problem solutions

for file in os.listdir(r'../instances'):
    instance_ = f"../instances/{file}"
    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
        for RM in [0.75, 1.0, 1.5, 3.0, 5.0]:
            for GM in [0.6, 0.7, 0.8, 0.9, 1.0]:
                run_solver_output(model='fixed', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
                run_solver_output(model='flex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)    
                for max_n_shift in range(2,5):
                    run_solver_output(model='partflex', instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)


in instance
in solver
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14578 rows, 21784 columns and 27490 nonzeros
Model fingerprint: 0xa1432774
Variable types: 14160 continuous, 7624 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 550.8400081
Found heuristic solution: objective 550.8399961
Presolve removed 14133 rows and 18353 columns
Presolve time: 0.04s
Presolved: 445 rows, 3431 columns, 7230 nonzeros
Found heuristic solution: objective 550.4400020
Variable types: 506 continuous, 2925 integer (6 binary)

Root relax

In [4]:
#Load and concat all raw scheduling solution

list_output = []

for filename in os.listdir(r'../results/results_output'):
    with open(f'../results/results_output/{filename}', 'r') as file:
        data = json.load(file)
        df_ = pd.DataFrame(data)
        list_output.append(df_)

df_output = pd.concat(list_output, ignore_index = True)
df_output.to_excel(r'../results/df_raw_scheduling_solution.xlsx', index = False)


In [ ]:
for file in os.listdir(r'../instances'):
    instance_ = f"../instances/{file}"

    run_solver_shift(model='fixed', instance=instance_, outsourcing_cost_multiplier=1.2, regional_multiplier=.75, global_multiplier=.6)
    run_solver_shift(model='flex', instance=instance_, outsourcing_cost_multiplier=1.2, regional_multiplier=.75, global_multiplier=.6)    
    for max_n_shift in range(2,5):
        run_solver_shift(model='partflex', instance=instance_, outsourcing_cost_multiplier=1.2, regional_multiplier=.75, global_multiplier=.6, max_n_shifts=max_n_shift)


In [8]:
filename = 'results_output_berlin_db=0.50_dt=uniform_oc=1.2_rm=0.75_gm=0.6_mu=2_model=partflex.json'

with open(f'../results/results_output/{filename}', 'r') as file:
    data = json.load(file)
    df_ = pd.DataFrame(data)
    df_.sort_values(by = ['region','theta'], inplace = True)

# #get the regions and make a dictionary
# dict_shifts = {}
# for region in df_['region'].unique().tolist():
#     dict_shifts[region] = {}
#     dict_shifts[region]['shifts_start'] = {0:0,1:4}
#     dict_shifts[region]['shifts_end'] = {0:4,1:8}

# print(dict_shifts)

df_.sort_values(by = ['region','theta'], inplace = True)
df_ = df_[df_['zminus__a_theta']==1]
df_.drop_duplicates(subset = ['region','theta'], inplace = True)
dict_shifts = {}
for region in df_['region'].unique().tolist():
    dict_shifts[region] = {}
    dict_shifts[region]['shifts_start'] = {}
    dict_shifts[region]['shifts_end'] = {}
    for index_, start_theta in enumerate(df_[df_['region']==region]['theta'].tolist()):
        dict_shifts[region]['shifts_start'][index_] = start_theta
        dict_shifts[region]['shifts_end'][index_] = start_theta + 4

print(dict_shifts)

{0: {'shifts_start': {0: 0}, 'shifts_end': {0: 4}}, 1: {'shifts_start': {0: 0}, 'shifts_end': {0: 4}}, 2: {'shifts_start': {0: 0}, 'shifts_end': {0: 4}}}


In [ ]:

level = 'region' #this can be shifts on the region level or global level
city = 'berlin'
demand_baseline = 0.5
demand_type = 'peak'
outsourcing_cost_multiplier = 1.2
regional_multiplier = 0.75
global_multiplier = 0.6
model = 'fixed' #can be ('fixed','flex','partflex')
max_n_shift = 2 #when model = 'partflex' can be 2, 3, or 4

def load_shift(level, city, demand_baseline, demand_type, outsourcing_cost_multiplier, regional_multiplier, global_multiplier, model, max_n_shifts):
    #fixed or flex
    if model in ['fixed','flex']:
        shift_file = f'../shifts/{city}_db={demand_baseline}_dt={demand_type}_oc={outsourcing_cost_multiplier}_rm={regional_multiplier}_gm={global_multiplier}_model={model}.json'
    else:
        #partflex
        shift_file = f'../shifts/{city}_db={demand_baseline}_dt={demand_type}_oc={outsourcing_cost_multiplier}_rm={regional_multiplier}_gm={global_multiplier}_mu={max_n_shifts}__model={model}.json'

    #load file
    with open(shift_file, 'r') as file:
        data = json.load(file)

    if level == 'region':
        #return dictionary of dictionaries
        #each key is a region, first element is dictionary of shift_start, second element is shift_end

        print("placeholder")
        return data
    else:
        df_ = 
        #returns dictionary of shifts
        print("placeholder")

    return dict_    
